In [ ]:
import pandas as pd
import httpx
import asyncio
import json
from tqdm import tqdm

In [ ]:
df_cm = pd.read_csv("nt_collection_master_202308221024.csv", encoding="utf-8")

In [ ]:
df_migration = df_cm.loc[df_cm["collection_id"].str.contains("0x")]
df_migration

In [ ]:
df_solana = df_cm.loc[~df_cm["collection_id"].str.contains("0x")]

In [ ]:
df_solana.loc[df_solana["curation"].notnull()]

In [ ]:
async def _get_request(url, headers):
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=headers)
            json_obj = json.loads(json.dumps(response.json()))

    except Exception as ex:
        print(ex)

    return json_obj


res = []
chain_domains = ["api", "api-bsc", "api-polygon"]
urls = [
    f"https://{chain_domain}.reservoir.tools/collections/v6"
    for chain_domain in chain_domains
]
headers = {
    "accept": "application/json",
    "x-api-key": "",
}

for row in tqdm(df_cm.to_dict("records")):
    if "0x" in row["collection_id"]:
        res.append(row)
    else:
        # print(row)
        resps = await asyncio.gather(
            *[_get_request(f"{url}?slug={row['slug']}", headers) for url in urls]
        )

        for resp in resps:
            if not resp["collections"] == []:
                if resp["collections"][0]["id"] in ["", None]:
                    print(row["slug"])
                else:
                    row["collection_id"] = resp["collections"][0]["id"]
                    res.append(row)

df_new_cm = pd.DataFrame(res)
df_new_cm.to_csv("nt_collection_master_20230822T110300.csv", index=False)

In [ ]:
df_new_cm.loc[
    df_new_cm["collection_id"].isin(
        list(
            df_cm["collection_id"]
            .loc[~df_cm["collection_id"].str.contains("0x")]
            .to_dict()
            .values()
        )
    )
]

In [ ]:
df_new_cm.loc[
    df_new_cm["slug"].isin(
        list(
            df_cm["slug"]
            .loc[~df_cm["collection_id"].str.contains("0x")]
            .to_dict()
            .values()
        )
    )
]

In [ ]:
df_found_collection_id = df_new_cm.loc[
    df_new_cm["slug"].isin(
        list(
            df_cm["slug"]
            .loc[~df_cm["collection_id"].str.contains("0x")]
            .to_dict()
            .values()
        )
    )
]

In [ ]:
list(df_cm["slug"].loc[~df_cm["collection_id"].str.contains("0x")].to_dict().values())

In [ ]:
df_cm.loc[~df_cm["collection_id"].str.contains("0x")]

In [ ]:
df_col = pd.read_csv("collection_table_202308221450.csv", encoding="utf-8")
df_col

In [ ]:
df_col.loc[df_col["collection_name"].notnull()]

In [ ]:
len(
    df_col["collection_name"]
    .loc[df_col["collection_name"].notnull()]
    .to_dict()
    .values()
)

In [ ]:
from collections import Counter

collection_name_conut = Counter(
    df_col["collection_name"]
    .loc[df_col["collection_name"].notnull()]
    .to_dict()
    .values()
)

len([1 for _ in collection_name_conut if collection_name_conut[_] > 1])

In [ ]:
collection_name_map = {}

for row in tqdm(df_col.to_dict("records")):
    if row["collection_name"] not in collection_name_map:
        collection_name_map[row["collection_name"]] = [row["collection_id"]]
    elif row["collection_id"] not in collection_name_map[row["collection_name"]]:
        collection_name_map[row["collection_name"]].append(row["collection_id"])

In [ ]:
cnt = 0

for x in collection_name_map:
    if len(collection_name_map[x]) > 1:
        cnt += 1

cnt